### Visualise threedimodel

In this notebook we'll make an interactive map of a threedimodel using [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/index.html).
To run this notebook locally make sure you have it installed.


As always, we first create a api-client with our authentication details.

In [ ]:
import requests
import json
from getpass import getpass
from threedi_api_client.api import ThreediApi
from threedi_api_client.openapi.api.v3_api import V3Api
from ipyleaflet import Map, GeoJSON

In [ ]:
# Provide authentication details
API_HOST = "https://api.3di.live"
PERSONAL_API_KEY = getpass("Personal API token")  # https://management.3di.live/personal_api_keys

config = {
    "THREEDI_API_HOST": API_HOST,
    "THREEDI_API_PERSONAL_API_TOKEN": PERSONAL_API_KEY
}

api_client: V3Api = ThreediApi(config=config)

And the threedi-model we are interested in

In [ ]:
threedi_model = api_client.threedimodels_list(name__icontains='v2_bergermeer').results[0]

print(threedi_model)

Then we display all the cells of the model on a leaflet map:

In [ ]:
extent_coordinates = threedi_model.extent_two_d.coordinates

center = (extent_coordinates[0][1] + extent_coordinates[1][1]) / 2.0, (extent_coordinates[0][0] + extent_coordinates[1][0]) / 2.0
m = Map(center=center, zoom=13)

geojson_cells_download = api_client.threedimodels_geojson_cells_download(threedi_model.id)
r = requests.get(geojson_cells_download.get_url)
geojson_data = json.loads(r.text)

geo_json = GeoJSON(
    data=geojson_data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    }
)

m.add_layer(geo_json)

display(m)